# 📘 Example: Database Builder with Python

In this notebook, we demonstrate how to use the **Database builder** API of FloodAdapt to build a FloodAdapt database in a new location! For this, we will use an example area in Charleston, South Carolina, for which we have already generated a SFINCS and a Delft-FIAT model.

In order to use the **DatabaseBuilder** of FloodAdapt a set of **configuration** parameters are needed. The **configuration** parameters can be divided to **mandatory** and **optional** ones. Using only the mandatory parameters (i.e., baseline FloodAdapt configuration) will result in a simple but functional version of FloodAdapt. By adding optional parameters to your configuration, you can create a more advanced FloodAdapt database with additional features. If you want to learn more about the configuration parameters, please refer to the [Database-Builder](../../../4_system_setup/database.qmd) of the Setup Guide in the documentation.

The configuration can be either created through available FloodAdapt classes or can be parsed as a simple dictionary. We advice you to work with the FloodAdapt classes, since this can avoid using wrong parameter names or values with the help of type hinting. 

## Import libraries

In [ ]:
# Import packages
import pandas as pd
import geopandas as gpd
from pathlib import Path
from hydromt_fiat.fiat import FiatModel
from hydromt_sfincs.sfincs import SfincsModel
from flood_adapt import FloodAdapt, Settings
from flood_adapt import unit_system as us
from flood_adapt import database_builder as db
from flood_adapt.objects.forcing import TideGaugeSource
from flood_adapt.config.hazard import ObsPointModel, SlrScenariosModel, FloodModel
%matplotlib inline

The study area is in **Charleston, South Carolina**, a coastal city on the East Coast of the United States. To run this notebook, we have already prepared a SFINCS model and a Delft-FIAT model for this area. Both these models are meant for demonstration purposes only. 

In this notebook we will go through all the mandatory and optional configuration parameters to create a database.

# Mandatory configuration parameters

## 💾 Database save path

First, we need to define the path where the database will be saved. This is done by defining the `database_path` attribute.

In [ ]:
# Define the static data folder
STATIC_DATA_DIR = Path("../../../_data/examples/static-data/1_DatabaseBuilder").resolve()

# Where the database will be stored
database_path = (STATIC_DATA_DIR / "Database").as_posix()

## 🌐 Overland SFINCS model

One of the mandatory inputs for a FloodAdapt database is an overland SFINCS model. Let's first inspect the extents of our overland SFINCS model, by loading the model with the HydroMT-SFINCS plugin.

In [ ]:
# Get the path of the SFINCS overland model
fn_sfincs = STATIC_DATA_DIR  / "overland"
# Use HydroMT-SFINCS to read the SFINCS model
sfincs = SfincsModel(root=str(fn_sfincs), mode="r")
sfincs.read()
# Get the extent of the SFINCS model
gdf = sfincs.region[["geometry"]]
gdf["name"] = "SFINCS Model Extent"
# Make a map of the SFINCS model extent
gdf.explore(
    style_kwds={"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.2},
    tiles="CartoDB positron",
    column="name",
    legend=True,
    legend_kwds={"caption": "Region"}
)

The SFINCS model is specified using the `sfincs_overland` attribute, which is a `FloodModel` class that includes the path to the SFINCS model, defined by the attribute `name` and the vertical reference that the model has, defined by `reference`. The SFINCS model was build with elevation data in the **NAVD88** vertical reference system, so we set the `reference` to `NAVD88`.

In [ ]:
# Define the overland SFINCS model path and vertical reference
sfincs_overland = FloodModel(
    name=(STATIC_DATA_DIR / "overland").as_posix(),
    reference="NAVD88" 
)  

## 🏠 Delft-FIAT model

Another mandatory input is the Delft-FIAT model. We can inspect the exposure objects (buildings and roads) of the Delft-FIAT model, by loading the model with the HydroMT-FIAT plugin.

In [ ]:
# Get the path of the FIAT model
fn_fiat = STATIC_DATA_DIR  / "fiat"
# Read the FIAT model using HydroMT-FIAT
fiat = FiatModel(root=str(fn_fiat), mode="r")
fiat.read()
# Get the geodataframe with exposure data
gdf = fiat.exposure.get_full_gdf(fiat.exposure.exposure_db)
# Plot the region and the secondary_object_types of the exposure data
gdf.explore(
    column="primary_object_type", 
    name="Exposure types",
    tiles="CartoDB positron"
)

The Delft-FIAT model is simply specified using the `fiat` attribute, which points to the path of the Delft-FIAT model.

In [ ]:
# Define the FIAT model path
fiat = (STATIC_DATA_DIR / "fiat").as_posix()

## 📏 Define the unit system

Then, a `unit_system` needs to be specified, which can be either `metric` or `imperial`. The unit_system will determine the default units used in the database. Since for our example we are in U.S., we will use the `imperial` unit system.

In [ ]:
# Define the unit system for the database
unit_system = db.UnitSystems.imperial 

## 🗺️ GUI map color scaling

For the output visualizations, scaling values need to be specified for each output type, using the `gui` attribute. These values will be used to scale the colors of the map visualizations in the GUI. There are four types of outputs that can be visualized in the GUI: **flood_depth, aggregated damages, footprint damages, and benefits**. For each of these outputs, we can specify the maximum values for the color scaling using the `max_flood_depth`, `max_aggr_dmg`, `max_footprint_dmg`, and `max_benefits` attributes, respectively.

In [ ]:
# Define the max values for each type of layer in the GUI
gui = db.GuiConfigModel(
    max_flood_depth=5,
    max_aggr_dmg=1e6,
    max_footprint_dmg=250000,
    max_benefits=5e6,
) 

# Additional configuration parameters

## 🎲 Probabilistic event set and risk calculations

We can add a probabilistic event set by providing the path to an existing event set with the attribute `probabilistic_set`. This will enable us to run risk and benefit scenarios in FloodAdapt (see [Risk and benefit analysis](../../../4_system_setup/index.qmd#Risk-and-benefit-analysis)). 

In case we provide a probabilistic event set to enable risk calculations, we can also specify the return periods that will be calculated from the event set in FloodAdapt during risk scenario runs. The default values are [1, 2, 5, 10, 25, 50, 100] years, but you can specify any other set of values with the `return_periods` attribute.

In [ ]:
 # Path to the prepared probabilistic set
probabilistic_set = str(STATIC_DATA_DIR  / "test_set")
# Here we just use the standard return periods
return_periods = [1, 2, 5, 10, 25, 50, 100] 

## 🌀 SFINCS offshore model

If we have a SFINCS offshore model we can also pass this into the configuration with the `sfincs_offshore` attribute in the same way as the overland SFINCS model. This will allow us to run extra types of events (see [Simulating hurricane events and ‘ungauged’ historical events](../../../4_system_setup/index.qmd#Simulating-hurricane-events-and-'ungauged'-historical-events)). Let's first visualize the SFINCS offshore model to see its extent.

In [ ]:
# Use HydroMT-SFINCS to read the SFINCS model
off_sfincs_path = (STATIC_DATA_DIR / "offshore").as_posix()
sfincs = SfincsModel(root=off_sfincs_path, mode="r")
sfincs.read()
# Get the extent of the SFINCS model
gdf = sfincs.region[["geometry"]]
gdf["name"] = "offshore SFINCS Model Extent"
# Make a map of the SFINCS model extent
gdf.explore(
    style_kwds={"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.2},
    tiles="CartoDB positron",
    column="name",
    legend=True,
    legend_kwds={"caption": "Region"}
)

Similarly, to the onshore SFINCS model, we can use a FloodModel class to define the path with the attribute `name` and its vertical reference with the attribute `reference` (which for an offshore models is typically 'MSL'). In case a correction is needed to bring MSL to present day conditions (see [Sea level offset for offshore simulations](../../../2_technical_docs/EventScenario.qmd#Sea-level-offset-for-offshore-simulations)), the `vertical_offset` attribute can be used to specify the correction.

In [ ]:
# Add the SFINCS offshore model
sfincs_offshore = FloodModel(
    name=off_sfincs_path,
    reference="MSL",
    vertical_offset=us.UnitfulLength(
        value=0.33, units=us.UnitTypesLength.feet # in this case we found from observations that there is an offset of 0.33 feet
    )
)

## 🌪️ Historical hurricanes

If we have an offshore SFINCS model, we can run historical hurricanes as well if we are in a hurricane prone area. The configuration for running hurricanes or not, is set with the `cyclones` attribute, which in case we are in an area were hurricanes are not relevant we could turn to `False`. If this is set to `True` (which is the default value), the `cyclone_basin` attribute can be used to define the oceanic basin. The `Basins` class can be used to check the available basins. In the case of Charleston we are going to use `NA` - for North Atlantic. If this is not specified, all global basins will be downloaded.

In [ ]:
# Add cyclone tracks
cyclones = True
cyclone_basin = db.Basins.NA

## 🕒  Tide gauge data

If there are water level observations from a close by tide gauge we can add them in the database, so they can directly be used during event creation (see [Downloading historical water levels](../../../4_system_setup/index.qmd#downloading-historical-water-levels)), by using the `tide_gauge` attribute. 

With the `source` attribute assigned to `file`, and the use of the `file` attribute to define the path to a csv file with the tide gauge data, we can directly use the tide gauge data in the database. The vertical reference of the tide gauge data can be defined by the `ref` attribute. The CSV file should have two columns; the first contains a ‘datetime’ in the format DD/MM/YYYY HH:MM and the second column contains the water levels relative to the vertical reference defined.

In U.S., instead of manually providing a file, we can choose `db.TideGaugeSource.noaa_coops` as the `source` attribute, to find the closest tide gauge from the **NOOAA COOPS** tide gauge network. To avoid using a stations that is really far away, we can also specify a `max_distance` attribute, which will be used to filter the stations. If no station is found within the specified distance, the tide gauge data will not be added to the database. A set of water level references from this station will be added to the database as well. These include **"MLLW", "MHHW", "NAVD88", "MSL"**. The default reference of the observation is `MLLW`, which can be changed with the `ref` attribute.

In our case we will use the NOAA COOPS tide gauge data with a limit of 100 miles and we will keep the default reference of `MLLW`.

In [ ]:
# Add tide gauge
tide_gauge = db.TideGaugeConfigModel(
    source=TideGaugeSource.noaa_coops,
    max_distance=us.UnitfulLength(
        value=100, units=us.UnitTypesLength.miles
    ),
)

## 📍 Observation points

By using the attribute `obs_points` we can add a list of observation points for which we will extract timeseries of water levels as an output of our event scenarios. We can add a list of `ObsPointModel` objects. Each of these objects must have a `name` and a `lat` and `lon` attribute. The `description` is optional.

Keep in mind that if a tide gauge station is added to the database, it will be automatically added as an observation point as well if it falls within the overland SFINCS domain.

In [ ]:
# Add observation points
obs_point = [
    ObsPointModel(
        name="Ashley_river",
        description="Ashley River - James Island Expy",
        lat=32.7765,
        lon=-79.9543,
    ),
]

## 📈 Sea Level Rise (SLR) scenarios

We can add sea level rise scenarios to be used in the projections of FloodAdapt, by using the `slr_scenarios` attribute, which should be a `SlrScenariosModel` object, with a `file` attribute pointing to a csv file with the columns:  **year, unit, scenario_1, scenario_2, ..., scenario_n**, and a `relative_to_year` attribute, which indicate the year relative to which these scenarios should be translated, when used in FloodAdapt.

Here we have created a slr scenario csv file like this already. Let's have a quick look in what the csv file looks like.

In [ ]:
slr_csv = (STATIC_DATA_DIR  / "slr.csv").as_posix()
pd.read_csv(slr_csv)

In [ ]:
# Add SLR scenarios
slr_scenarios = SlrScenariosModel(
    file=slr_csv,
    relative_to_year=2020,
)

## 🧍‍♂️🧍‍♀️ Social Vulnerability Index (SVI)

A social vulnerability (SVI) layer can be added to the database for extra infographics related to who is impacted. This can be done with the `svi` attribute which is a `db.SviConfigModel` object. The path to a geospatial file with the SVI layer is provided with the `file` attribute, the `field_name` attribute defines the column name within the spatial file with the SVI value and the `threshold` defines the threshold value for the SVI, which distinguishes between vulnerable and non-vulnerable areas. 

In our case we have already clipped an SVI layer (from https://www.atsdr.cdc.gov/place-health/php/svi/svi-data-documentation-download.html) to the Charleston area, so we can use it directly. Let's have a quick look in what the SVI layer looks like.

In [ ]:
svi_path = (STATIC_DATA_DIR / "svi.gpkg").as_posix()
svi_layer = gpd.read_file(svi_path)
# Make a map of the SVI layer
svi_layer.explore(
    column="SVI",
    name="Social Vulnerability Index (SVI)",
    tiles="CartoDB positron",
    cmap="RdBu_r",
    scheme=None,
    style_kwds={"color": "black", "weight": 0.5, "fillOpacity": 0.7},
    legend=True,
    legend_kwds={"caption": "SVI (0.5=center)"},
    categorical=False,
    center=0.5
)

Now, let's create the SVI configuration object, using the `SviConfigModel` class.

In [ ]:
# Add social vulnerability index
svi = db.SviConfigModel(
    file=svi_path,
    field_name="SVI",
    threshold=0.5,
)

## 🌊 Base Flood Elevation (BFE)

A base flood elevation (BFE) can be added to the database which allows users to elevate homes relative to this layer. This can be done with the `bfe` attribute which is a `db.SpatialJoinModel` object. The path to the geospatial vector file with the BFE layer is provided with the `file` attribute, the `field_name` attribute defines the column name within the spatial file with the BFE value.

In our case we have already created some dummy data, so we can use it directly. Let's have a quick look in what the BFE layer looks like.

In [ ]:
bfe_path = (STATIC_DATA_DIR / "bfe.gpkg").as_posix()
bfe_layer = gpd.read_file(bfe_path)
# Make a map of the BFE layer
bfe_layer_valid = bfe_layer[bfe_layer["STATIC_BFE"] != -9999]
bfe_layer_valid.explore(
    column="STATIC_BFE",
    name="Base Flood Elevation (BFE) - feet",
    tiles="CartoDB positron",
    cmap="Blues",
    scheme=None,
    style_kwds={"color": "black", "weight": 0.5, "fillOpacity": 0.7},
    legend=True,
    categorical=False,
)

Let's create the BFE configuration object, using the `SpatialJoinModel` class.

In [ ]:
# Add base flood elevation
bfe = db.SpatialJoinModel(
    file=bfe_path,
    name="bfe",
    field_name="STATIC_BFE",
)

## 🔲 Aggregation areas

A list of different types of aggregation levels that can be included in the metrics aggregation can be defined by the `aggregation_areas` attribute. Each entry in the list can be defined by the `SpatialJoinModel` class, which has a `name` attribute to specify the shown name, a `file` attribute to specify the location of the geospatial vector file. The `name` is used to identify the aggregation area in the database, while the `file` is the path to a geospatial vector file with the aggregation area polygons and a `field_name` attribute to specify the column name within the spatial file with the aggregation area names.

Keep in mind that the aggregation areas specified in the database builder configuration are additional to any aggregation areas already defined in the Delft-FIAT model.

In [ ]:
aggr_path = (STATIC_DATA_DIR / "subdivision.gpkg").as_posix()
# Here we can visualize in a map one of the aggragetion levels
aggr = gpd.read_file(aggr_path)
# Make a map of the aggregation level 1 layer
aggr.explore(
    column="SUDIV_UNIQUE",
    name="Aggregation Level: Subdivision",
    tiles="CartoDB positron",
    style_kwds={"color": "black", "weight": 0.5, "fillOpacity": 0.7},
    legend=True,
    legend_kwds={"caption": "Aggregation Level 1"},
)

We can now add the aggregation level to the configuration, using the `SpatialJoinModel` class.

In [ ]:
# Add aggregation areas
aggregation_areas = [
    db.SpatialJoinModel(
        name="Subdivision",
        file=aggr_path,
        field_name="SUDIV_UNIQUE",
    )
]

## 🏘️🛣️ Exposure Objects types

The location of the exposure objects (buildings and roads) in Delft-FIAT is defined by different geospatial vector files. By default, the name of the buildings file is set to `buildings` and the roads file is set to `roads`. If this is not the case these values can be changed using the `fiat_buildings_name` and `fiat_roads_name` attributes, respectively.

FloodAdapt works with roads as polygons, so if the `roads` file is in line format, we can use the `road_width` attribute to define the width of the road. This will be used to create a polygon representation of the roads in the database. By default this is set to `5.0` meters, but it can be changed to any other value.

In [ ]:
# In our case we keep the default values for the exposure options
fiat_buildings_name = "buildings"
fiat_roads_name = "roads"
road_width = us.UnitfulLength(value=5, units=us.UnitTypesLength.meters)

## 🏢 Building Footprints

Buildings in FIAT exposure can be represent as points. Then we want to aggregate these to building footprints, which are polygons. This can be done by using the `building_footprints` attribute, which is a `db.SpatialJoinModel` object. The `file` attribute defines the path to the geospatial vector file with the building footprints and the `field_name` attribute defines the column name within the spatial file with the building footprint names. If we don't provide a building footprints file, the building footprints will be downloaded from the OpenStreetMap (OSM) database.

In [ ]:
# Visualize building footprints in a single color
building_fp_path = (STATIC_DATA_DIR / "FEMA_building_footprints.gpkg").as_posix()
building_fp_gdf = gpd.read_file(building_fp_path)
building_fp_gdf.explore(
    name="Building Footprints",
    tiles="CartoDB positron",
    style_kwds={"color": "black", "weight": 0.5, "fillColor": "#3182bd", "fillOpacity": 0.7},
    legend=False
)

We can now define the building footprints configuration object, using the `SpatialJoinModel` class.

In [ ]:
building_footprints = db.SpatialJoinModel(
    file=building_fp_path,
    field_name="BUILD_ID", # unique identifier for the building footprints
)

# Create the database configurations

Now, that we created all the individual objects we can compile them in a `ConfigModel` object. As mentioned using only the mandatory attributes we can already build a functional FloodAdapt database. However, we can also add the optional attributes to create a more advanced FloodAdapt database. Here we are going to create two configurations, one with only the mandatory attributes and one with all the optional attributes as well.

A mandatory attribute is the `name` of the database, which will be the unique identifier of the database. The `database_path` attribute defines the path where these database will be saved.

In [ ]:
# Compile ConfigModel with only the mandatory attributes
config_model_basic = db.ConfigModel(
    name = "charleston_example_basic",
    database_path=database_path,
    unit_system=unit_system,  
    gui=gui, 
    sfincs_overland=sfincs_overland,
    fiat=fiat, 
)

In [ ]:
# Compile ConfigModel with all the optional attributes
config_model_advanced = db.ConfigModel(
    name = "charleston_example_advanced",
    database_path=database_path,
    unit_system=unit_system,  
    gui=gui, 
    sfincs_overland=sfincs_overland,
    fiat=fiat, 
    probabilistic_set=probabilistic_set,
    return_periods=return_periods,
    sfincs_offshore=sfincs_offshore,
    slr_scenarios=slr_scenarios,
    tide_gauge=tide_gauge,
    cyclones=cyclones,
    cyclone_basin=cyclone_basin,
    obs_point=obs_point,
    aggregation_areas=aggregation_areas,
    building_footprints=building_footprints,
    fiat_buildings_name=fiat_buildings_name,
    fiat_roads_name=fiat_roads_name,
    bfe=bfe,
    svi=svi,
    road_width=road_width,
)

# Run the Database Builder

We are now ready to run the Database Builder with the configurations that we just created above. When running the database builder, all the steps and inputs that are used are logged and printed to the console. Moreover, a log file is saved in the database path.

Let's first create the basic database.

In [ ]:
db.create_database(config=config_model_basic, overwrite=True)

Now, we can create the advanced database with all the optional attributes.

In [ ]:
db.create_database(config=config_model_advanced, overwrite=True)

# Reading-in the FloodAdapt database
Now that we built the database we can open it and continue to work with it.  

Let's open the advanced database that we just created and verify it can be loaded correctly.

In [ ]:
settings = Settings(
    DATABASE_ROOT=Path(STATIC_DATA_DIR / "Database").resolve(),
    DATABASE_NAME="charleston_example_advanced"
)
settings.export_to_env()
fa = FloodAdapt(database_path=settings.database_path)